<a href="https://colab.research.google.com/github/JingQian87/AppliedDL/blob/master/Lecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading要求：

## L1: 
* DLwP: C1&2
* DL: C1
* Python DS Handbook

## L2: 
* DLwP: C3
* DL: 4.1,4.2,4.3,5.1,5.2

## L3: 
* DLwP: C4&5
* DL: 9

## L4: 
* DL: 4.1, 4.3

## L5: project相关

## L6: 
* Deep Learning: 7.12 (Dropout), 8.3 (Basic Algorithms); 8.4 (Parameter Initialization Strategies); 8.5 (Algorithms with Adaptive Learning Rates)

## L7:  C7

## L1: 
* DLwP: C1&2

# DLwP C1
## 1.1
**Machine learning**: find useful representations of input data. 各种变形，比如linear projections, translations, nonlinear operations, etc.

** Deep Learning**, 又名layered representations learning, hierarchical representations learning.
* a multistage way to learn data representations
* depth = \# layers in the model
* 每一层的weights决定了transformation 如何参数化。我们需要找到合适的weights使得input map correctly to targets.
* 衡量方式：loss function/ objective function, compute distance between prediction & true value. 得到*loss score*.
* Optimizer: 通过backpropogation 去调整weights, 从而降低loss score.






## 1.2. Brief history of ML
1. Probabilistic Modeling. 
Eg. Naive Bayes, Logistic regression.

2. Kernel Methods.
Eg: SVM

3. Decision Tree. Random Forest. Gradient Boosting Machines-> 最好的适合nonperceptual data的算法

DL优点：不用烦feature engineering.







# DLwP C2